In [1]:
import numpy as np
import pandas as pd
from scipy.stats import itemfreq

import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
# reading data
data_path = './data/'

predicted_demand = pd.read_csv('./predicted_demand.csv')
# predicted_demand = pd.read_csv(data_path+'train/historical_demand.csv')
delivery_cost = pd.read_csv(data_path+'train/delivery_cost.csv')
production_capacity = pd.read_csv(data_path+'train/production_capacity.csv')

In [3]:
regions = np.unique(predicted_demand['Region'])
products = np.unique(predicted_demand['Product_ID'])

In [10]:
def get_suited_plant_from_region_delivery_cost(prod, region_delivery_cost):
    # right now we are considering that region_delivery_cost will contain only
    # one region's cost
    regions = region_delivery_cost['Region'].tolist()
    delivery_cost_arr = region_delivery_cost['Delivery_cost'].tolist()
    delivery_cost_arr = sorted(delivery_cost_arr)
    result = pd.DataFrame(columns=['Plant', 'Line', 'Product', 'Capacity'])
    for deli_cost in delivery_cost_arr:
        region_deli_cost = region_delivery_cost.loc[\
                   (region_delivery_cost['Delivery_cost'] == deli_cost)]
        capacity = production_capacity.loc[\
                            (production_capacity['Product'] == prod)]
        if capacity['Capacity'].sum() > 0:
            result.append(capacity)
    return result

In [8]:
def find_optimal_solution():
    # Calculate the product demand overall
    for i in range(len(products)):
        freq = predicted_demand.\
            loc[(predicted_demand['Product_ID'] == products[i])]
        prod_freq[i] = freq['Demand'].sum()
    # sort the prod_freq
    sorted_freq = sorted(prod_freq)
    # reverse it
    sorted_freq.reverse()
    print("Top tend demanding products ",\
          [products[prod_freq.index(freq)] for freq in sorted_freq][:10])
    
    # loop through the prod_freq
    for freq in sorted_freq:
        prod = products[prod_freq.index(freq)]
        # get where the demand is
        demand_region_df = predicted_demand.loc[\
                                (predicted_demand['Product_ID'] == prod) \
                                        & (predicted_demand['Demand'] > 0)]
        # here we may have to find regions list from demand_region and then itereate over them
        # get delivery cost for demand_region
        demand_region_name = demand_region_df['Region'].iloc[0]
        region_delivery_cost = delivery_cost.loc[(delivery_cost['Region'] == demand_region_name)]
        